# Main Results - Changing EV Adoption Levels

Developed by Siobhan Powell, 2021. 


In [1]:
import os
os.chdir('..')

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import scipy
import os.path
import pandas
from simple_dispatch import generatorData
from simple_dispatch import bidStack
from simple_dispatch import dispatch
from simple_dispatch import generatorDataShort
from future_grid import FutureDemand
import matplotlib
import shutil
from simple_dispatch import StorageModel
import copy

In [3]:
os.mkdir('Results/StorageCurves')

# Load generator model

In [5]:
gd_short = pickle.load(open('IntermediateOutputs/generator_data_short_WECC_2019.obj', 'rb'))

# Dispatch without storage before to calculate storage needed

In [13]:
max_rate = 10000 # assumes average 4 hour storage - note high uncertainty
cap = 40000

maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20211119.csv', index_col=0)
maxpen_results4 = pd.read_csv('Results/max_penetration_levels_1h_storagebefore_bigger_20211119.csv', index_col=0)

maxpen_results3 = pd.read_csv('Results/max_penetration_levels_1h_bau_20211119.csv', index_col=0)
maxpen_results5 = pd.read_csv('Results/max_penetration_levels_1h_storagebefore_bigger_bau_20211119.csv', index_col=0)


# calculate breaking point for each case
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone'}

fuel = 1
solar = 2.5
wind = 2.5

for penlevel in np.arange(0.2, .95, 0.1):
    for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']:
        # no workplace control
        ev_workplace_bool=False
        for i, ev_timers in enumerate(['', '_midnighttimer', '_NoTimers']):
            ev_scenario_date = '20211119'
            # Load and set-up future grid and demand model 
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
            save_str = 'Results/StorageCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(np.round(penlevel, 2))
            str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
            max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
            print(save_str)
            # Check for overgeneration
            grid.check_overgeneration(save_str=save_str)
            # Run dispatch
            grid.run_dispatch(max_penlevel, save_str, result_date='20211119')  

        # workplace control
        ev_workplace_bool=True
        for wp_control in ['avgem', 'minpeak']:
            ev_timers = '_NoTimers'
            ev_scenario_date = '20211119'
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
            save_str = 'Results/StorageCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(np.round(penlevel, 2))
            grid.check_overgeneration(save_str=save_str)

            str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
            max_penlevel = maxpen_results2.loc[timer_names[ev_timers], str1]
            print(save_str)
            grid.run_dispatch(max_penlevel, save_str, result_date='20211119')  


    # BAU Case
    ev_scenario = 'BusinessAsUsual'
    ev_workplace_bool=True
    ev_timers=''
    ev_scenario_date='20211119'
    wp_control = 'minpeak'
    grid = FutureGrid(gd_short)
    grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                            ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                            ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)
    save_str = 'Results/StorageCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(np.round(penlevel, 2))
    grid.check_overgeneration(save_str=save_str)

    str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
    max_penlevel = maxpen_results3.loc['TimersMixed', str1]
    print(save_str)
    grid.run_dispatch(max_penlevel, save_str, result_date='20211119') 

Results/StorageCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers9pm_noWPcontrol_penlevel0.2
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/StorageCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers12am_noWPcontrol_penlevel0.2
1.9% Complete
3.8% Complete
5.8% Comp

38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/StorageCurves/fuel1_solar2.5_wind2.5_LowHome_HighWork_Timers9pm_noWPcontrol_penlevel0.2
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Co

80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/StorageCurves/fuel1_solar2.5_wind2.5_LowHome_LowWork_TimersNone_WPcontrol_minpeak_penlevel0.2
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96

15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/StorageCurves/fuel1_solar2.5_wind2.5_HighHome_TimersNone_WPcontrol_avgem_penlevel0.3
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Compl

59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Results/StorageCurves/fuel1_solar2.5_wind2.5_LowHome_LowWork_TimersNone_noWPcontrol_penlevel0.3
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Co

...

21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 6617
Storage Capacity:  26468
Results/StorageCurves/fuel1_solar2.5_wind2.5_HighHome_Timers9pm_noWPcontrol_penlevel0.8
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1%

50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 1153
Storage Capacity:  4613
Results/StorageCurves/fuel1_solar2.5_wind2.5_LowHome_HighWork_TimersNone_noWPcontrol_penlevel0.8
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Comple

86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 3792
Storage Capacity:  15170
Results/StorageCurves/fuel1_solar2.5_wind2.5_BusinessAsUsual_TimersMixed_WPcontrol_minpeak_penlevel0.8
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5%

3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 3960
Storage Capacity:  15842
Results/StorageCurves/fuel1_solar2.5_wind2.5_HighHome_TimersNone_WPcontrol_avgem_penlevel0.9
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5

42.3% Complete
44.2% Complete
46.2% Complete
48.1% Complete
50.0% Complete
51.9% Complete
53.8% Complete
55.8% Complete
57.7% Complete
59.6% Complete
61.5% Complete
63.5% Complete
65.4% Complete
67.3% Complete
69.2% Complete
71.2% Complete
73.1% Complete
75.0% Complete
76.9% Complete
78.8% Complete
80.8% Complete
82.7% Complete
84.6% Complete
86.5% Complete
88.5% Complete
90.4% Complete
92.3% Complete
94.2% Complete
96.2% Complete
98.1% Complete
100.0% Complete
Solving optimization.
Storage Rate Result: 2018
Storage Capacity:  8072
Results/StorageCurves/fuel1_solar2.5_wind2.5_LowHome_LowWork_Timers12am_noWPcontrol_penlevel0.9
----Capacity too low----
Try with storage:
1.9% Complete
3.8% Complete
5.8% Complete
7.7% Complete
9.6% Complete
11.5% Complete
13.5% Complete
15.4% Complete
17.3% Complete
19.2% Complete
21.2% Complete
23.1% Complete
25.0% Complete
26.9% Complete
28.8% Complete
30.8% Complete
32.7% Complete
34.6% Complete
36.5% Complete
38.5% Complete
40.4% Complete
42.3% Complet

# Storage before - don't need to run dispatch, but run calculation up until dispatch

In [14]:
os.mkdir('Results/ImpactCurves')

In [15]:
max_rate = 10000 # assumes average 4 hour storage - note high uncertainty
cap = 40000

maxpen_results2 = pd.read_csv('Results/max_penetration_levels_1h_20211119.csv', index_col=0)
maxpen_results4 = pd.read_csv('Results/max_penetration_levels_1h_storagebefore_bigger_20211119.csv', index_col=0)

maxpen_results3 = pd.read_csv('Results/max_penetration_levels_1h_bau_20211119.csv', index_col=0)
maxpen_results5 = pd.read_csv('Results/max_penetration_levels_1h_storagebefore_bigger_bau_20211119.csv', index_col=0)


# calculate breaking point for each case
timer_names = {'':'Timers9pm', '_midnighttimer':'Timers12am', '_NoTimers':'TimersNone'}
maxpen_results = {}

fuel = 1
solar = 2.5
wind = 2.5


for penlevel in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:#np.arange(0.1, .95, 0.1):
    for ev_scenario in ['UniversalHome', 'HighHome', 'LowHome_HighWork', 'LowHome_LowWork']:
        # no workplace control
        ev_workplace_bool=False
        for i, ev_timers in enumerate(['', '_midnighttimer', '_NoTimers']):
            ev_scenario_date = '20211119'
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                                                ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control='', ev_workplace_bool=False, evs_bool=True, 
                                                                                ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

            save_str = 'Results/ImpactCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_noWPcontrol'+'_penlevel'+str(penlevel)+'_storagebefore'
            grid.check_overgeneration(save_str=save_str)
            # Run storage before dispatch: 
            grid.run_storage_before_capacitydispatch(cap, max_rate)
            grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20211119'+'.csv')

            str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_noWPcontrol'
            max_penlevel = maxpen_results4.loc[timer_names[ev_timers], str1]
            print(save_str)

            grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
            grid.future.demand.to_csv(save_str+'_futuredemand_df_20211119.csv')
#             run_dispatch(gd_short_copy, future, max_penlevel, save_str, result_date='20211004') 

        # workplace control
        ev_workplace_bool=True
        for wp_control in ['avgem', 'minpeak']:
            ev_timers = '_NoTimers'
            ev_scenario_date = '20211119'
            grid = FutureGrid(gd_short)
            grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                        ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                        ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

            save_str = 'Results/ImpactCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+timer_names[ev_timers]+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
            grid.check_overgeneration(save_str=save_str)
            # Run storage before dispatch: 
            grid.run_storage_before_capacitydispatch(cap, max_rate)
            grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20211119'+'.csv')

            str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
            max_penlevel = maxpen_results4.loc[timer_names[ev_timers], str1]
            print(save_str)
            grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
            grid.future.demand.to_csv(save_str+'_futuredemand_df_20211119.csv')


    # BAU Case
    ev_scenario = 'BusinessAsUsual'
    ev_workplace_bool=True
    ev_timers=''
    ev_scenario_date='20211119'
    wp_control = 'minpeak'
    grid = FutureGrid(gd_short)
    grid.set_up_scenario(year=2030, solar=solar, wind=wind, fuel=fuel, ev_scenario=ev_scenario, 
                                                    ev_timers=ev_timers, ev_pen=penlevel, ev_workplace_control=wp_control, ev_workplace_bool=True, evs_bool=True, 
                                                    ev_scenario_date=ev_scenario_date, weekend_date=ev_scenario_date,  weekend_timers=ev_timers)

    save_str = 'Results/ImpactCurves/fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_'+'TimersMixed'+'_WPcontrol_'+wp_control+'_penlevel'+str(penlevel)+'_storagebefore'
    grid.check_overgeneration(save_str=save_str)
    # Run storage before dispatch: 
    grid.run_storage_before_capacitydispatch(cap, max_rate)
    grid.storage.df.to_csv(save_str+'_storagebeforedf_'+'20211119'+'.csv')

    str1 = 'fuel'+str(fuel)+'_solar'+str(solar)+'_wind'+str(wind)+'_'+ev_scenario+'_WPcontrol_'+wp_control
    max_penlevel = maxpen_results5.loc['TimersMixed', str1]
    print(save_str)
    grid.future.demand['demand'] = np.copy(grid.storage.df.comb_demand_after_storage.values)
    grid.future.demand.to_csv(save_str+'_futuredemand_df_20211119.csv')
#     run_dispatch(gd_short_copy, future, max_penlevel, save_str, result_date='20211119') 

Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers9pm_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers12am_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_minpeak_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers9pm_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers12am_noWPcontrol_penlevel0.1_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_TimersNone_noWPcontrol_p

Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers9pm_noWPcontrol_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers12am_noWPcontrol_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_noWPcontrol_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_minpeak_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers9pm_noWPcontrol_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers12am_noWPcontrol_penlevel0.4_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_TimersNone_noWPcontrol_p

Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers9pm_noWPcontrol_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_Timers12am_noWPcontrol_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_noWPcontrol_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_avgem_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_UniversalHome_TimersNone_WPcontrol_minpeak_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers9pm_noWPcontrol_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_Timers12am_noWPcontrol_penlevel0.7_storagebefore
Solving optimization.
Results/ImpactCurves/fuel1_solar2.5_wind2.5_HighHome_TimersNone_noWPcontrol_p